In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,mls,mls
band,[5],3c
commitnumber,1013f91,a06b618
conc,-,-
dv,-,0.001
klin,-,5e-25
molecule,{'h2o': 'atmpro'},h2o
ng_adju,-,[0]
ng_refs,-,[5]
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD               CLIRAD `coolr_bands`          
            layer        coolrg                layer    coolrg
pressure                                                      
0.000312        1 -7.332137e-06                    1 -0.000013
0.000750        2 -6.714662e-06                    2 -0.000013
0.001052        3 -5.351271e-06                    3 -0.000012
0.001476        4 -3.845447e-06                    4 -0.000012
0.002070        5 -2.269908e-06                    5 -0.000011
0.002904        6 -6.299917e-07                    6 -0.000011
0.004074        7  1.077172e-06                    7 -0.000010
0.005714        8  2.855360e-06                    8 -0.000010
0.008015        9  4.706660e-06                    9 -0.000009
0.011243       10  6.632064e-06                   10 -0.000008
0.015771       11  8.637204e-06                   11 -0.000008
0.022122       12  1.071992e-05                   12 -0.000007
0.031031       13  1.288702e-05                   13 -0.000006
0.043528       14  1.528539e-05                   14 -0.000005
0.061057       15  2.289568e-05                   15 -0.000003
0.085645       16  4.808131e-05                   16  0.000006
0.120136       17  9.844485e-05                   17  0.000025
0.168516       18  1.739997e-04                   18  0.000053
0.236378       19  2.830424e-04                   19  0.000093
0.331549       20  4.375364e-04                   20  0.000151
0.465100       21  6.528289e-04                   21  0.000231
0.652400       22  9.458306e-04                   22  0.000349
0.915100       23  1.245885e-03                   23  0.000470
1.283650       24  1.269348e-03                   24  0.000479
1.800600       25  1.022228e-03                   25  0.000385
2.525700       26  7.620041e-04                   26  0.000287
3.542800       27  5.506584e-04                   27  0.000206
4.969550       28  3.902550e-04                   28  0.000145
6.970850       29  2.744161e-04                   29  0.000100
9.778100       30  1.907055e-04                   30  0.000068
13.715850      31  1.273329e-04                   31  0.000042
19.239350      32  8.000926e-05                   32  0.000023
26.987250      33  4.954378e-05                   33  0.000010
37.855300      34  3.079623e-05                   34  0.000002
53.100050      35  1.604577e-05                   35 -0.000005
73.887500      36  4.160113e-06                   36 -0.000010
97.662500      37 -1.988900e-06                   37 -0.000014
121.437500     38 -4.260249e-06                   38 -0.000015
145.212500     39 -6.223905e-06                   39 -0.000016
168.987500     40 -6.059462e-06                   40 -0.000017
192.762500     41  4.943875e-06                   41 -0.000017
216.537500     42  9.424350e-05                   42  0.000004
240.312500     43  4.353080e-04                   43  0.000165
264.087500     44  1.309218e-03                   44  0.000686
287.862500     45  3.046075e-03                   45  0.001916
311.637500     46  5.078867e-03                   46  0.003652
335.412500     47  7.449703e-03                   47  0.005877
359.187500     48  1.039904e-02                   48  0.008796
382.962500     49  1.392995e-02                   49  0.012416
406.737500     50  1.801749e-02                   50  0.016588
430.512500     51  2.261848e-02                   51  0.021116
454.287500     52  2.717921e-02                   52  0.025496
478.062500     53  3.194901e-02                   53  0.029964
501.837500     54  3.721105e-02                   54  0.034853
525.612500     55  4.300520e-02                   55  0.040192
549.387500     56  4.937696e-02                   56  0.046398
573.162500     57  5.643169e-02                   57  0.053606
596.937500     58  6.696224e-02                   58  0.064462
620.712500     59  7.832457e-02                   59  0.076325
644.487500     60  8.987480e-02                   60  0.088183
668.262500     61  1.016594e-01              

# Fluxes by Layer

CRD                              CLIRAD  \
                        flug          fldg      fnetg       flug   
pressure    level                                                  
0.000000    1     -30.744163  0.000000e+00 -30.744163 -30.805987   
0.000624    2     -30.744163  4.166112e-09 -30.744163 -30.805987   
0.000876    3     -30.744163  5.865661e-09 -30.744163 -30.805987   
0.001229    4     -30.744163  8.314810e-09 -30.744163 -30.805987   
0.001723    5     -30.744163  1.185102e-08 -30.744163 -30.805987   
0.002417    6     -30.744163  1.695572e-08 -30.744163 -30.805987   
0.003391    7     -30.744163  2.432492e-08 -30.744163 -30.805987   
0.004757    8     -30.744163  3.496248e-08 -30.744163 -30.805987   
0.006672    9     -30.744163  5.031598e-08 -30.744163 -30.805987   
0.009359    10    -30.744163  7.247545e-08 -30.744163 -30.805987   
0.013128    11    -30.744163  1.044554e-07 -30.744163 -30.805987   
0.018415    12    -30.744163  1.506057e-07 -30.744163 -30.805988   
0.025830    13    -30.744163  2.172011e-07 -30.744163 -30.805988   
0.036232    14    -30.744163  3.132954e-07 -30.744163 -30.805988   
0.050823    15    -30.744163  4.522025e-07 -30.744163 -30.805988   
0.071291    16    -30.744164  6.649804e-07 -30.744163 -30.805988   
0.100000    17    -30.744164  1.041610e-06 -30.744163 -30.805988   
0.140271    18    -30.744164  1.771039e-06 -30.744162 -30.805988   
0.196760    19    -30.744164  3.184764e-06 -30.744161 -30.805988   
0.275997    20    -30.744165  5.909233e-06 -30.744158 -30.805989   
0.387100    21    -30.744164  1.112799e-05 -30.744153 -30.805989   
0.543100    22    -30.744162  2.107803e-05 -30.744141 -30.805988   
0.761700    23    -30.744156  3.983688e-05 -30.744116 -30.805987   
1.068500    24    -30.744144  7.286107e-05 -30.744071 -30.805984   
1.498800    25    -30.744126  1.197493e-04 -30.744006 -30.805979   
2.102400    26    -30.744108  1.743675e-04 -30.743933 -30.805974   
2.949000    27    -30.744091  2.343081e-04 -30.743857 -30.805972   
4.136600    28    -30.744078  2.990095e-04 -30.743779 -30.805971   
5.802500    29    -30.744071  3.686502e-04 -30.743703 -30.805974   
8.139200    30    -30.744070  4.443581e-04 -30.743626 -30.805982   
11.417000   31    -30.744080  5.276806e-04 -30.743552 -30.805997   
16.014700   32    -30.744102  6.195412e-04 -30.743483 -30.806020   
22.464000   33    -30.744143  7.212829e-04 -30.743422 -30.806056   
31.510500   34    -30.744208  8.388495e-04 -30.743369 -30.806108   
44.200100   35    -30.744304  9.821127e-04 -30.743322 -30.806186   
62.000000   36    -30.744448  1.159300e-03 -30.743289 -30.806300   
85.775000   37    -30.744648  1.370926e-03 -30.743277 -30.806461   
109.550000  38    -30.744856  1.572664e-03 -30.743283 -30.806631   
133.325000  39    -30.745069  1.775061e-03 -30.743295 -30.806809   
157.100000  40    -30.745291  1.978658e-03 -30.743312 -30.806993   
180.875000  41    -30.745529  2.199076e-03 -30.743329 -30.807192   
204.650000  42    -30.745906  2.590521e-03 -30.743315 -30.807511   
228.425000  43    -30.746939  3.889312e-03 -30.743050 -30.808411   
252.200000  44    -30.749307  7.483471e-03 -30.741824 -30.810533   
275.975000  45    -30.754030  1.589331e-02 -30.738136 -30.814857   
299.750000  46    -30.762480  3.292240e-02 -30.729558 -30.822710   
323.525000  47    -30.774248  5.899283e-02 -30.715255 -30.833762   
347.300000  48    -30.789229  9.495604e-02 -30.694273 -30.847872   
371.075000  49    -30.807833  1.428472e-01 -30.664987 -30.865468   
394.850000  50    -30.830443  2.046876e-01 -30.625755 -30.886976   
418.625000  51    -30.857404  2.823911e-01 -30.575013 -30.912717   
442.400000  52    -30.889053  3.777411e-01 -30.511312 -30.942951   
466.175000  53    -30.924967  4.902009e-01 -30.434766 -30.977326   
489.950000  54    -30.964996  6.202078e-01 -30.344788 -31.015730   
513.725000  55    -31.009278  7.692883e-01 -30.239989 -31.058350   
537.500000  56    -31.057907  9.390337e-01 -30.118873 -31.105122   
561.275000  5

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')